In [ ]:
# Code to read and process IMD soundings for ASTraL IOP1
# Alex Kinsella, June 2024, modified from code by Simon de Szoeke for ASraL Pilot, June 2023

In [1]:
#cd("~/Projects/ASTRAL/soundings/src/")

# activate the environment
# an environment is just a path with Manifest.toml and Project.toml
using Pkg; Pkg.activate("..")

using Revise
#using Regex
using CSV, DataFrames
using Interpolations
using Dates
using PyPlot
using NCDatasets

  Activating project at `~/Documents/WHOI/Projects/ASTraL/2024_IOP1/ASTRAL-soundings`


In [2]:
reldatapath = "../data/"
CamelStations = ["Ahmedabad","Bhubaneswar","Chennai","Karaikal","Kochi","Kolkata","Minicoy","PortBlair","Pune","SantaCruz","Visakhapatnam"]

"Parse the various date formats in the file names. Only works for 3-letter month (or May)."
function imd_file_date(fn)
    if fn[1:6]=="43150-" && all(isnumeric, fn[[7:14; 16:17]]) # 43150 Visakhap. WMO station ID
        dt = Dates.DateTime(fn[7:17], "yyyymmdd-HH")
    elseif fn[10:12]=="VSK" && all(isnumeric, fn[[1:2; 6:9]])  # Visakhap.
        dt = Dates.DateTime(fn[1:9], "ddUUUyyyy")
    elseif all(isnumeric, fn[[1:8; 10:11]])
        dt = Dates.DateTime(fn[1:11], "yyyymmdd-HH")
    elseif all(isnumeric, fn[[1:4; 6:7; 9:10; 12:13; 15:16; 18:19]])
        dt = Dates.DateTime(fn[1:13], "yyyy-mm-dd_HH")
    else
        # expect undelimited 3-letter month
        if all(isnumeric, fn[[1:2; 6:7]])
            if isletter(fn[8])
                dt = DateTime( 2000+year(DateTime(fn[6:7], "yy")),
                    month(DateTime(fn[3:5], "U")),
                    day(DateTime(fn[1:2], "d")) )
            elseif all(isnumeric, fn[8:9])
                dt = DateTime( year(DateTime(fn[6:9], "yyyy")),
                    month(DateTime(fn[3:5], "U")),
                    day(DateTime(fn[1:2], "d")) )
            end
        end
    end
    return dt
end


"count unique soundings from each station, return maximum"
function count_max_unique_soundings(stationnames)
    nts = zeros(Int32, length(stationnames))
    
    for (sj, station) in enumerate(stationnames)
        # read standard named files in the station's directory
        files = soundingfiles(station)
        # deduplicate sounding times
        fdt = imd_file_date.(files)
        fdt12 = round.(fdt, Dates.Hour(12))
        ui = uniqueidx(fdt12)
        fdt[ui] # exact datetimes with duplicates removed
        ord = sortperm(fdt[ui]) # guarantees ascending order
        nts[sj] = length(ui)
        # @show station, nts[sj]
    end
    
    nt = maximum(nts)
end

soundingfiles(stationdir; reldatapath=reldatapath) = filter(x -> occursin(r"(?i)(Standard_Summary|Standard-Summary|Standard Summary).*(\.txt)$(?-i)",x),
                   readdir(joinpath(reldatapath, stationdir)) ) 

uniqueidx(v) = unique(i -> v[i], eachindex(v))

# Interpolation methods
# Pressure is irregular and different for each sounding time.

"dv converts Datetimes to numeric values for Interpolations."
dv(dt, offset=DateTime(2019,3,1)) = Dates.value(dt - offset)

# interpolation functions
function itp_sparse(x::Vector, p::Vector)
    # knot-vectors must be unique and sorted in increasing order
    ind = sortperm(p)
    ii = isfinite.(p[ind])
    extrapolate( interpolate( (p[ind][ii],), x[ind][ii], Gridded(Linear()) ), NaN )
end

"Filter missing data from the input sounding before interpolating."
function itp(x::Vector, p::Vector)
    # knot-vectors must be unique and sorted in increasing order
    ind = sortperm(p)
    ii = isfinite.(p[ind] .+ x[ind]) # .& .!ismissing.(p[ind] .+ x[ind])
    extrapolate( interpolate( (p[ind][ii],), x[ind][ii], Gridded(Linear()) ), NaN )
end

"rectangular interval averaging onto a decreasing pressure coordinate"
function pavg(x::Vector, p::Vector; plevs=1020.0:-5.0:0.0)
    s1 = zeros(Float64, size(plevs))
    # s2 = zeros(Float64, size(p))
    ns = zeros(Float64, size(plevs))
    
    for i in eachindex(p)
        j = findfirst(x -> x < p[i], plevs) # j indexes plevs
        s1[j] += isfinite(x[i]) ? x[i] : 0
        # s2[j] += x[i]^2
        ns[j] += 1.0*isfinite(x[i])
    end
    
    s1 ./ ns # mean
end

# pressure levels to interploate to
dp = 5.0
plevs = 1020.0:-dp:0.0

"single-character replacer handles tabs and \u00b0 degree character °"
stripjunk(f) = IOBuffer(replace(read(f), UInt8('\t') => UInt8(' '), UInt16('\u00b0') => UInt8('d')) )

"read the file and return a DataFrame"
#function read_sonde( station, file; header=hdrs(station,file) )
function read_sonde( file )
    open( file, "r" ) do io
        lines = readlines( io )
        hdr = findfirst(x-> ~isempty(x)&&length(x)>4&&strip(x)[1:5]=="Time,", lines)
        if isnothing(hdr)
            hdr = length(lines)+1
        end
        footskip=0
        if lines[end][1]=='-'
            footskip=1
            if lines[end-1][1]=='-'
                footskip=2
            end
        end
        CSV.read( stripjunk( file ),
            DataFrame, delim=",", stripwhitespace=true, ignorerepeated=true, header=hdr[1],footerskip=footskip)
    end
end

"extract column data by name"
function get_sounding_cols(df, station)
    if isempty(df)
        # p = [missing]
        # T = [missing]
        # Td = [missing]
        # rh= [missing]
        # z = [missing]
        # wspd = [missing]
        # wdir = [missing]
        p = []
        T = []
        Td = []
        rh= []
        z = []
        wspd = []
        wdir = []
    else
        if in(station, ["mangalore", "bhubaneswar", "visakhapatnam", "portblair"])
            if "P(hPa)" in names(df)
                p = df[!,"P(hPa)"][:]
                T = df[!,"T(C)"][:]
                Td = df[!,"Dew(C)"][:]
                rh = df[!,"U(%)"][:]
                z = df[!,"Geo(gpm)"][:]
                wspd = df[!,"Wspd(m/s)"][:]
                wdir = df[!,"Wdir(d)"][:]
            elseif "p(hPa)" in names(df) # slightly different keys
                p = df[!,"p(hPa)"][:]
                rh = df[!,"U(%)"][:]
                z = df[!,"Geo(gpm)"][:]
                wspd = df[!,"Wspd(m/s)"][:]
                if "T(dC)" in names(df)
                    T = df[!,"T(dC)"][:]
                    Td = df[!,"Dew(dC)"][:]
                    wdir = df[!,"Wdir(d)"][:]
                elseif "T(Deg.C)" in names(df) # another variation
                    T = df[!,"T(Deg.C)"][:]
                    Td = df[!,"Dew(Deg.C)"][:]
                    wdir = df[!,"Wdir(Deg)"][:]
                end
            end
        else # slightly different keys
            p = df[!,"p(hPa)"][:]
            rh = df[!,"U(%)"][:]
            z = df[!,"Geo(gpm)"][:]
            wspd = df[!,"Wspd(m/s)"][:]
            if "T(dC)" in names(df)
                T = df[!,"T(dC)"][:]
                Td = df[!,"Dew(dC)"][:]
                wdir = df[!,"Wdir(d)"][:]
            elseif "T(Deg.C)" in names(df) # another variation
                T = df[!,"T(Deg.C)"][:]
                Td = df[!,"Dew(Deg.C)"][:]
                wdir = df[!,"Wdir(Deg)"][:]
            end

        end
    end
    u = -wspd .* sin.(pi/180.0*wdir)
    v = -wspd .* cos.(pi/180.0*wdir)
    
    return p, T,Td, rh,z, wspd,wdir, u,v
end

"read and grid IMD soundings to pressure levels."
function grid_imd_soundings!( time, T, Td, rh, z, u, v; 
                              plevs=plevs, stationnames=stationnames, reldatapath=reldatapath )
    for (sj, station) in enumerate(stationnames)
        @show station
        # read standard named files in the station's directory
        files = soundingfiles(station)
        # sj += 1 # gridded station index
        # deduplicate sounding times
        fdt = imd_file_date.(files)
        fdt12 = round.(fdt, Dates.Hour(12))
        ui = uniqueidx(fdt12)
        fdt[ui] # exact datetimes with duplicates removed
        ord = sortperm(fdt[ui]) # guarantees ascending order
        #collect(zip( files[ui[ord]]], fdt[ui[ord]] ))
        
        tj = 0
        for ti in ui[ord]
            fn = joinpath(reldatapath, station, files[ti])
            @show fn
            df = read_sonde( joinpath(reldatapath, station, files[ti]) )
            if !isempty(df)
                # extract ungridded hi-res data
                p_, T_,Td_, rh_,z_, wspd_,wdir_, u_,v_ = get_sounding_cols(df, station)
                # avg variables at the next time level
                tj += 1
                # update arrays in place
                time[sj,tj] = fdt[ti]
                T[  :, sj, tj] .= pavg(T_ , p_, plevs=plevs)
                Td[ :, sj, tj] .= pavg(Td_, p_, plevs=plevs)
                rh[ :, sj, tj] .= pavg(rh_, p_, plevs=plevs)
                z[  :, sj, tj] .= pavg(z_ , p_, plevs=plevs)
                u[  :, sj, tj] .= pavg(u_ , p_, plevs=plevs)
                v[  :, sj, tj] .= pavg(v_ , p_, plevs=plevs)
            else
                tj += 1
                time[sj,tj] =fdt[ti]
                # leave an empty sounding when missing
            end
        end
    end
    return # nothing
end

"output days from offset"
mpldate(timestamp, offset=DateTime(2023,4,30,0,0,0)) = Dates.value(timestamp-offset)/1000/60/60/24
#invmpldate( mlpd, offset=DateTime(2023,4,30,0,0,0) ) = mpldate+Dates.value(offset)/1000/60/60/24
mplday(d; offset=Date(2023,4,30)) = day(Day(d) + offset) # ticks

"Update plot all u,rh sounding timeheight profiles and put to APL."
function plot_timeheights( time, T, Td, rh, z, u, v )
    # count times at each station
    nts = sum(time.>DateTime(1000), dims=2)
    for sj in 1:size(time)[1]
        @show sj
        @show mpldate.(time[sj,1:nts[sj]])
        clf()
        subplot(2,1,1)
        pcolormesh( mpldate.(time[sj,1:nts[sj]]), plevs, u[:,sj,1:nts[sj]], vmin=-10, vmax=10, cmap=ColorMap("RdYlBu_r") )
        ylim(1000, 100)
        ylabel("u (m/s)")
        xtl,~ = xticks()
        xticks( xtl, mplday.(xtl,offset=Date(2023,4,30)) )
        colorbar()
        title(CamelStations[sj])
    
        subplot(2,1,2)
        pcolormesh( mpldate.(time[sj,1:nts[sj]]), plevs, rh[:,sj,1:nts[sj]], cmap=ColorMap("RdYlBu_r"))
        ylim(1000, 100)
        ylabel("RH (%)")
        xtl,~ = xticks()
        xticks( xtl, mplday.(xtl,offset=Date(2023,4,30)) )
        colorbar()
        xlabel(Dates.format(time[sj,1], "yyyy U"))
    
        figfile = "../plot/$(CamelStations[sj])_u_rh.svg"
        savefig( figfile )
        #put_file_apl( figfile )
    end
end

"Write out pressure-gridded sonde data. Push new files to APL. Let old files stand."
function write_new_grid_sondes( time, T, Td, rh, z, u, v; stationnames )
    # count times at each station
    nts = sum(time.>DateTime(1000), dims=2)
    for (sj, station) in enumerate(stationnames)
        for tj in 1:nts[sj]
            fileout = joinpath("../data/grid_sondes/", Dates.format(time[sj,tj], "yyyymmdd-HH")*"_$(CamelStations[sj]).csv")
            fileoutnc = joinpath("../data/grid_sondes/", Dates.format(time[sj,tj], "yyyymmdd-HH")*"_$(CamelStations[sj]).nc")
            if !isfile(fileout)
                # @show fileout
                dfo = DataFrame(  p=plevs,
                                  T=T[  :, sj, tj],
                                  Td=Td[:, sj, tj],
                                  rh=rh[:, sj, tj],
                                  z=z[  :, sj, tj],
                                  u=u[  :, sj, tj],
                                  v=v[  :, sj, tj]  )
                CSV.write( fileout, dfo )

                ds = NCDataset(fileoutnc,"c")
                ds.attrib["time"] = Dates.format(time[sj,tj], "yyyymmdd-HH")

                defDim(ds,"p",length(plevs))
                
                
                # Define the variables
                Tvar=defVar(ds,"T",T[  :, sj, tj],("p",))
                Tdvar=defVar(ds,"Td",Td[:, sj, tj],("p",))
                rhvar=defVar(ds,"rh",rh[:, sj, tj],("p",))
                zvar=defVar(ds,"z",z[  :, sj, tj],("p",))
                uvar=defVar(ds,"u",u[  :, sj, tj],("p",))
                vvar=defVar(ds,"v",v[  :, sj, tj],("p",))    
                pvar=defVar(ds,"p",plevs,("p",))             
                
                # write attributes
                Tvar.attrib["units"] = "deg C"
                Tdvar.attrib["units"] = "deg C"
                rhvar.attrib["units"] = "%"
                zvar.attrib["units"] = "m"
                uvar.attrib["units"] = "m/s"
                vvar.attrib["units"] = "m/s"
                pvar.attrib["units"] = "mb"

                close(ds)

                #put_file_apl( fileout )
            end
        end
    end
end


write_new_grid_sondes

In [4]:
# do
# a big loop to read all sounding files, interpolate data to consistent levels
# output gridded files, and make plots

# reiterate parameters for paths, names, and gridding
reldatapath = "../data/"
plevs = 1020.0:-5.0:0.0
CamelStations = ["Ahmedabad","Bhubaneswar","Chennai","Karaikal","Kochi","Kolkata","Mangalore","Minicoy","PortBlair","Pune","SantaCruz","Visakhapatnam"]
stationnames = lowercase.(CamelStations)

# allocate gridded data arrays (pressure, location, launch time)
nt = count_max_unique_soundings(stationnames)
# nt = 80 # allow for longest station mangalore?(37), extend & trim off extra later
times = zeros(DateTime, length(stationnames), nt)
T  = NaN.+zeros(Float64, length(plevs), length(stationnames), nt)
Td = NaN.+zeros(Float64, length(plevs), length(stationnames), nt)
rh = NaN.+zeros(Float64, length(plevs), length(stationnames), nt)
z  = NaN.+zeros(Float64, length(plevs), length(stationnames), nt)
u  = NaN.+zeros(Float64, length(plevs), length(stationnames), nt)
v  = NaN.+zeros(Float64, length(plevs), length(stationnames), nt)

grid_imd_soundings!( times, T, Td, rh, z, u, v; plevs=plevs, stationnames=stationnames, reldatapath=reldatapath )

#plot_timeheights( times, T, Td, rh, z, u, v )
write_new_grid_sondes( times, T, Td, rh, z, u, v; stationnames )

station = "ahmedabad"
fn = "../data/ahmedabad/20240415-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240416-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240417-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240418-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240419-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240420-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240421-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240422-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240423-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240424-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240425-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240426-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240427-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240428-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240429-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240430-00-Standard_Summary.txt"
fn = "../data/ahmedabad/20240501-00-Standard_Summa

In [41]:
"Update plot all u,rh sounding timeheight profiles and put to APL."
function plot_timeheights( time, T, Td, rh, z, u, v )
    # count times at each station
    nts = sum(time.>DateTime(1000), dims=2)
    for sj in 1:size(time)[1]
        @show sj
        @show mpldate.(time[sj,1:nts[sj]])
        clf()
        subplot(2,1,1)
        pcolormesh( mpldate.(time[sj,1:nts[sj]]), plevs, u[:,sj,1:nts[sj]], vmin=-10, vmax=10, cmap=ColorMap("RdYlBu_r") )
        ylim(1000, 100)
        ylabel("u (m/s)")
        xtl,~ = xticks()
        xticks( xtl, mplday.(xtl,offset=Date(2023,4,30)) )
        colorbar()
        title(CamelStations[sj])
    
        subplot(2,1,2)
        pcolormesh( mpldate.(time[sj,1:nts[sj]]), plevs, rh[:,sj,1:nts[sj]], cmap=ColorMap("RdYlBu_r"))
        ylim(1000, 100)
        ylabel("RH (%)")
        xtl,~ = xticks()
        xticks( xtl, mplday.(xtl,offset=Date(2023,4,30)) )
        colorbar()
        xlabel(Dates.format(time[sj,1], "yyyy U"))
    
        figfile = "../plot/$(CamelStations[sj])_u_rh.svg"
        savefig( figfile )
        #put_file_apl( figfile )
    end
end

plot_timeheights